In [121]:
import pandas as pd
import random
import math
import io
import requests
import datetime
import time
from collections import deque

In [105]:
covid19 = pd.read_csv("/Users/sameeraturupu/Downloads/Safest-path-through-corona-affected-regions-master/Counties_info.csv")
# covid19 = pd.read_csv("Counties_info.csv")

In [106]:
covid19.head()

,Source,Destination,Time,Confirmed,2019 Census
0,Clayton,Dekalb,26,869,"292,256"
1,Clayton,Fayette,22,99,"114,421"
2,Clayton,Fulton,26,1467,"1,063,937"
3,Clayton,Henry,27,260,"234,561"
4,Cobb,Douglas,30,153,"146,343"


# Preparing the input dictionary for A* algorithm with neighbor and time information

In [107]:
input_dict = {}
for src in covid19['Source']:  
    input_dict[src.lower()] = []
    

for key in input_dict:
    for src,dest,time,conf,census in zip(covid19['Source'],covid19['Destination'],covid19['Time'],covid19['Confirmed'],covid19['2019 Census']):
        if(key == src.lower()):
            input_dict[key].append((dest.lower(),round(int(time)*(int(conf)/int(census.replace(',','')))*1000)))

In [108]:
print(input_dict)

{'clayton': [('dekalb', 77), ('fayette', 19), ('fulton', 36), ('henry', 30)], 'cobb': [('douglas', 31), ('fulton', 36)], 'coweta': [('fayette', 22), ('fulton', 66)], 'dekalb': [('clayton', 33), ('gwinnett', 19), ('fulton', 33), ('henry', 43)], 'douglas': [('cobb', 28), ('fulton', 41)], 'fulton': [('clayton', 32), ('cobb', 24), ('coweta', 33), ('dekalb', 71), ('douglas', 31), ('fayette', 36), ('gwinnett', 24)], 'gwinnett': [('dekalb', 77), ('fulton', 44)], 'fayette': [('clayton', 27), ('fulton', 58), ('coweta', 18)], 'henry': [('clayton', 33), ('dekalb', 116)]}


# Heuristic information for each county

In [109]:
h_org = {}
for dest, conf, census in zip (covid19['Destination'],covid19['Confirmed'],covid19['2019 Census']):
    h_org[dest.lower()] = round((int(conf)/int(census.replace(',','')))*1000)

In [110]:
h_org

{'dekalb': 3,
 'fayette': 1,
 'fulton': 1,
 'henry': 1,
 'douglas': 1,
 'clayton': 1,
 'gwinnett': 1,
 'cobb': 1,
 'coweta': 1}

# Getting most recent(5 days) COVID-19 information 

In [111]:
today = datetime.date.today().strftime('%m/%d/%Y')

date_1 = datetime.datetime.strptime(str(today), "%m/%d/%Y")
start_date = date_1 + datetime.timedelta(days=-6)
end_date = date_1 + datetime.timedelta(days=-1)

start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

start = datetime.datetime.strptime(start_date, "%Y-%m-%d")
end = datetime.datetime.strptime(end_date, "%Y-%m-%d")
date_array = (start + datetime.timedelta(days=x) for x in range(0, (end - start).days))
dates = []
for date_object in date_array:
    dates.append(str(date_object.strftime("%m-%d-%Y")))
print(dates)

['04-13-2020', '04-14-2020', '04-15-2020', '04-16-2020', '04-17-2020']


In [112]:
url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"
c = []
for date in dates:
    s = requests.get(url + date + ".csv").content
    c.append(pd.read_csv(io.StringIO(s.decode('utf-8'))))

# Heuristics for most recent COVID-19 information

In [113]:
h_upt = []
h1_intial = {}
for i in range(len(c)): 
    for county, state, confirmed in zip(c[i]['Admin2'], c[i]['Province_State'], c[i]['Confirmed']):
        if (state == 'Georgia'):
            if county.lower() in h_org:
                h1_intial[county.lower()] = confirmed
    h_upt.append(h1_intial)
    h1_intial = {}

In [114]:
h_upt

[{'clayton': 391,
  'cobb': 782,
  'coweta': 119,
  'dekalb': 980,
  'douglas': 167,
  'fayette': 105,
  'fulton': 1598,
  'gwinnett': 739,
  'henry': 280},
 {'clayton': 435,
  'cobb': 895,
  'coweta': 135,
  'dekalb': 1144,
  'douglas': 189,
  'fayette': 115,
  'fulton': 1812,
  'gwinnett': 815,
  'henry': 306},
 {'clayton': 456,
  'cobb': 924,
  'coweta': 136,
  'dekalb': 1191,
  'douglas': 197,
  'fayette': 120,
  'fulton': 1844,
  'gwinnett': 852,
  'henry': 321},
 {'clayton': 468,
  'cobb': 990,
  'coweta': 140,
  'dekalb': 1247,
  'douglas': 202,
  'fayette': 124,
  'fulton': 1929,
  'gwinnett': 896,
  'henry': 330},
 {'clayton': 491,
  'cobb': 1072,
  'coweta': 149,
  'dekalb': 1349,
  'douglas': 220,
  'fayette': 133,
  'fulton': 2025,
  'gwinnett': 1017,
  'henry': 344}]

In [115]:
heuristic_list = []
heuristic_dict = {}
for heuristic in h_upt:
    for county, census in zip(covid19['Destination'],covid19['2019 Census']):
        val = round((int(heuristic[county.lower()])/int(census.replace(',','')))* 1000)
        if heuristic[county.lower()] not in heuristic_dict:
            heuristic_dict[county.lower()] = val
    heuristic_list.append(heuristic_dict)
    heuristic_dict = {}

In [116]:
heuristic_list

[{'dekalb': 3,
  'fayette': 1,
  'fulton': 2,
  'henry': 1,
  'douglas': 1,
  'clayton': 1,
  'gwinnett': 1,
  'cobb': 1,
  'coweta': 1},
 {'dekalb': 4,
  'fayette': 1,
  'fulton': 2,
  'henry': 1,
  'douglas': 1,
  'clayton': 1,
  'gwinnett': 1,
  'cobb': 1,
  'coweta': 1},
 {'dekalb': 4,
  'fayette': 1,
  'fulton': 2,
  'henry': 1,
  'douglas': 1,
  'clayton': 2,
  'gwinnett': 1,
  'cobb': 1,
  'coweta': 1},
 {'dekalb': 4,
  'fayette': 1,
  'fulton': 2,
  'henry': 1,
  'douglas': 1,
  'clayton': 2,
  'gwinnett': 1,
  'cobb': 1,
  'coweta': 1},
 {'dekalb': 5,
  'fayette': 1,
  'fulton': 2,
  'henry': 1,
  'douglas': 2,
  'clayton': 2,
  'gwinnett': 1,
  'cobb': 1,
  'coweta': 1}]

# Working of A* Algorithm

In [124]:
class Graph:
    # example of adjacency list (or rather map)
    # adjacency_list = {
    # 'A': [('B', 1), ('C', 3), ('D', 7)],
    # 'B': [('D', 5)],
    # 'C': [('D', 12)]
    # }

    def __init__(self, adjacency_list):
        self.adjacency_list = adjacency_list

    def get_neighbors(self, v):
        return self.adjacency_list[v]

    # heuristic function with equal values for all nodes
    def h(self, n, H):
        return H[n]

    def a_star_algorithm(self, start_node, stop_node):
        # open_list is a list of nodes which have been visited, but who's neighbors
        # haven't all been inspected, starts off with the start node
        # closed_list is a list of nodes which have been visited
        # and who's neighbors have been inspected
        open_list = {start_node}
        closed_list = set([])

        # g contains current distances from start_node to all other nodes
        # the default value (if it's not found in the map) is +infinity
        g = {}
        H = h_org
        g[start_node] = 0

        # parents contains an adjacency map of all nodes
        parents = {start_node: start_node}
        
        while len(open_list) > 0:
            n = None
            n1 = None
            # find a node with the lowest value of f() - evaluation function
            for v in open_list:
                if n is None or g[v] + self.h(v, H) < g[n] + self.h(n, H):
                    n = v;
            H = random.choice(heuristic_list)
            if n is None:
                print('Path does not exist!')
                return None
            # if the current node is the stop_node
            # then we begin reconstructin the path from it to the start_node
            if n == stop_node:
                reconst_path = []

                while parents[n] != n:
                    reconst_path.append(n)
                    n = parents[n]

                reconst_path.append(start_node)

                reconst_path.reverse()

                print('Path found: {}'.format(reconst_path))
                return reconst_path

            # for all neighbors of the current node do
            for (m, weight) in self.get_neighbors(n):
                # if the current node isn't in both open_list and closed_list
                # add it to open_list and note n as it's parent
                if m not in open_list and m not in closed_list:
                    open_list.add(m)
                    parents[m] = n
                    g[m] = g[n] + weight
                # otherwise, check if it's quicker to first visit n, then m
                # and if it is, update parent data and g data
                # and if the node was in the closed_list, move it to open_list
                else:
                    if g[m] > g[n] + weight:
                        g[m] = g[n] + weight
                        parents[m] = n

                        if m in closed_list:
                            closed_list.remove(m)
                            open_list.add(m)

            # remove n from the open_list, and add it to closed_list
            # because all of his neighbors were inspected
            open_list.remove(n)
            closed_list.add(n)
        print('Path does not exist!')
        return None

In [125]:
graph1 = Graph(input_dict)
graph1.a_star_algorithm('cobb', 'coweta')

Path found: ['cobb', 'fulton', 'coweta']


['cobb', 'fulton', 'coweta']